# P1
Written by Babak Hosseini Parsa Ahmadi 810101609, Mohtasham 810101408 and Mahdy Naeini 810101536\
Data science project abou digikala mobile phones.

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import scipy.stats as stats
import numpy as np
import seaborn as sns

In [2]:
sub_features = {"وزن":"Weight"}

In [3]:
def make_new_csv(file_addr):
    with open(file_addr, mode='w') as mobiles_file:
        mobilesCSV = csv.writer(mobiles_file)
        mobilesCSV.writerow(list(sub_features.values()))

In [79]:
def write_to_csv(addr, driver,writer):
    driver.get("https://www.digikala.com/"+addr)
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight*0.7);')
    sleep(3)
    driver.find_elements(
        By.XPATH, "//*[contains(text(), 'مشاهده بیشتر')]")[-1].click()
    page = driver.page_source
    soup = bs(page, 'html.parser')
    global sub_features
    scraped_sub_features = [""]*len(sub_features)
    for i,feature in enumerate(sub_features.keys()):
        scraped_sub_features[i] = soup.find(
            string=feature).parent.parent.findAll('p')[-1].text
    writer.writerow(list(scraped_sub_features))

In [31]:
def scrape_all_mobiles(file_addr, number_of_mobiles):
    driver = webdriver.Chrome()
    mobiles=[]
    i=1
    while len(mobiles) < number_of_mobiles:
        driver.get(f'https://www.digikala.com/search/category-mobile-phone/product-list/?page={i}')
        sleep(1)
        page = driver.page_source
        soup = bs(page, 'html.parser')
        mobiles = soup.find_all(
            'div', {'class': 'product-list_ProductList__item__LiiNI'})
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        i+=1
    csv_file = open(file_addr, mode='a', encoding='utf-8')
    csv_writer=csv.writer(csv_file)
    for mobile in mobiles:
        write_to_csv(mobile.find('a')['href'], driver, csv_writer)
    driver.close()

In [6]:
FILE_ADDR='digikala_phones.csv'

In [7]:
make_new_csv(FILE_ADDR)

In [78]:
scrape_all_mobiles(FILE_ADDR,5)

/tmp/ipykernel_4322/1990730596.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  scraped_sub_features[i] = soup.find(


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (860, 4136)
  (Session info: chrome=122.0.6261.69)
Stacktrace:
#0 0x5e79b0b1ff33 <unknown>
#1 0x5e79b0817ce6 <unknown>
#2 0x5e79b0869a73 <unknown>
#3 0x5e79b086798e <unknown>
#4 0x5e79b0865367 <unknown>
#5 0x5e79b0864732 <unknown>
#6 0x5e79b0857d27 <unknown>
#7 0x5e79b0884eb2 <unknown>
#8 0x5e79b08576b8 <unknown>
#9 0x5e79b088507e <unknown>
#10 0x5e79b08a3899 <unknown>
#11 0x5e79b0884c53 <unknown>
#12 0x5e79b0855db3 <unknown>
#13 0x5e79b085677e <unknown>
#14 0x5e79b0ae586b <unknown>
#15 0x5e79b0ae9885 <unknown>
#16 0x5e79b0ad3181 <unknown>
#17 0x5e79b0aea412 <unknown>
#18 0x5e79b0ab725f <unknown>
#19 0x5e79b0b0e528 <unknown>
#20 0x5e79b0b0e723 <unknown>
#21 0x5e79b0b1f0e4 <unknown>
#22 0x7aae1e297b5a <unknown>
